<a href="https://colab.research.google.com/github/bortch/notebooks/blob/main/OCRizeMyPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr ocrmypdf tesseract-ocr-fra tesseract-ocr-nld
!pip install PyPDF2 Pillow pytesseract googletrans==4.0.0-rc1
!pip install pymupdf ocrmypdf

In [75]:
import io
import os
from google.colab import files
import fitz
import ocrmypdf

In [84]:
def ocr_the_page(page):
    """Extract the text from passed-in PDF page."""
    src = page.parent  # the page's document
    doc = fitz.open()  # make temporary 1-pager
    doc.insert_pdf(src, from_page=page.number, to_page=page.number)
    pdfbytes = doc.tobytes()
    inbytes = io.BytesIO(pdfbytes)  # transform to BytesIO object
    outbytes = io.BytesIO()  # let ocrmypdf store its result pdf here
    ocrmypdf.ocr(
        inbytes,  # input 1-pager
        outbytes,  # ouput 1-pager
        language="fra",  # modify as required e.g. ("eng", "ger","fra","nl")
        output_type="pdf",  # only need simple PDF format
        # add more paramneters, e.g. to enforce OCR-ing, etc., e.g.
        # force_ocr=True, redo_ocr=True
    )
    ocr_pdf = fitz.open("pdf", outbytes.getvalue())  # read output as fitz PDF
    text = ocr_pdf[0].get_text()  # ...and extract text from the page
    return text  # return it

In [ ]:
pdf_file = files.upload()

In [ ]:
filename = "my_uploaded_file.pdf" # change filename here
doc = fitz.open(filename)
print(doc.is_pdf)

In [ ]:
# output file
with open('out_%s.md' % filename, "w") as fp:
  textBuffer = ""
  for page in doc:
    fp.writelines('## %s\n' % page.number)
    textBuffer += ocr_the_page(page)
  fp.write(textBuffer)

In [ ]:
# clean directory
out_path = 'output/'
if os.path.exists(out_path):
  for file_name in os.listdir(out_path):
    file = out_path + file_name
    if os.path.isfile(file):
        print('Deleting file:', file)
        os.remove(file)

In [ ]:
# check if language is installed
# !apt-cache search tesseract-ocr